# Flask を使った簡易Webアプリ

まずは必要なライブラリをインストール

In [2]:
!pip install -r requirements.txt

     |████████████████████████████████| 102 kB 931 kB/s eta 0:00:01
     |████████████████████████████████| 224 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 6.6 MB/s  eta 0:00:01
     |████████████████████████████████| 59 kB 2.2 MB/s eta 0:00:01
You should consider upgrading via the '/Users/fujinohidenori/dev/project/.whisper_env/bin/python -m pip install --upgrade pip' command.


## その１　とりあえず音声ファイルをアップロードできるようにする．

In [42]:
%%writefile static/index.html

<!DOCTYPE html>
<html lang="ja">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Voice Chat App</title>
  </head>
  <body>
    <h1>Voice Chat App</h1>
    <button id="start">開始</button>
    <button id="stop" disabled>停止</button>
    <p><strong>文字起こし:</strong> <span id="transcription"></span></p>
    <p><strong>AIの応答:</strong> <span id="aiResponse"></span></p>

    <script>
      document.addEventListener("DOMContentLoaded", () => {
        const startButton = document.getElementById("start");
        const stopButton = document.getElementById("stop");
        const transcriptionElement = document.getElementById("transcription");
        const aiResponseElement = document.getElementById("aiResponse");
        let mediaRecorder;
        let audioChunks = [];

        startButton.addEventListener("click", async () => {
          const stream = await navigator.mediaDevices.getUserMedia({
            audio: true,
          });
          mediaRecorder = new MediaRecorder(stream);

          mediaRecorder.ondataavailable = (event) => {
            audioChunks.push(event.data);
          };

          mediaRecorder.onstop = async () => {
            const audioBlob = new Blob(audioChunks, { type: "audio/webm" });
            audioChunks = [];

            const formData = new FormData();
            formData.append("audio", audioBlob, "recording.webm");

            fetch("/upload", {
              method: "POST",
              body: formData,
            })
              .then((response) => response.json())
              .then((data) => {
                transcriptionElement.textContent =
                  data.text || "認識できませんでした。";
                aiResponseElement.textContent =
                  data.ai_response || "AIの応答なし。";
              })
              .catch((error) => {
                console.error("Upload failed:", error);
                transcriptionElement.textContent = "エラーが発生しました。";
                aiResponseElement.textContent = "";
              });
          };

          mediaRecorder.start();
          startButton.disabled = true;
          stopButton.disabled = false;
        });

        stopButton.addEventListener("click", () => {
          mediaRecorder.stop();
          startButton.disabled = false;
          stopButton.disabled = true;
        });
      });
    </script>
  </body>
</html>


Overwriting static/index.html


In [ ]:
%%writefile app01.py

from flask import Flask, request, jsonify, send_from_directory
import os

app = Flask(__name__, static_folder="static")  

@app.route("/")
def index():
    return send_from_directory("static", "index.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "audio" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["audio"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)
    
    text = "test"
    ai_response = "test"
    return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Overwriting app01.py


ふむ．とりあえず音声ファイル.webmはアップロードできるようになった．
ただspeech_recgnitionではwebmは受け入れないので，wavファイルに変える必要がある

## その２　アップロードした音声ファイルをWavファイルに変換

In [ ]:
%%writefile app02.py

from flask import Flask, request, jsonify, render_template
import os
import subprocess

app = Flask(__name__)

@app.route("/")
def index():
    return render_template("index.html")  # フロントエンドのHTMLを表示

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "audio" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["audio"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)
    convert_webm_to_wav(audio_path, "uploads/output.wav")
    
    text = "test"
    ai_response = "test"
    return jsonify({"text": text, "ai_response": ai_response})


def convert_webm_to_wav(input_path, output_path):
    command = [
        "ffmpeg",
        "-i", input_path,  # 入力ファイル
        "-ar", "16000",  # サンプリングレート 16kHz
        "-ac", "1",  # モノラル変換
        "-preset", "ultrafast",  # 速度最優先
        output_path
    ]
    subprocess.run(command, check=True)

# 使い方

if __name__ == "__main__":
    app.run(debug=True)


Writing app02.py


変換に少々時間取られるな．．．

## その３　アップロードを直接Wavファイルにする．

変換に少々時間がかかるのが気になるので，アップロードの段階で直接Wavファイルをアップできないか探ってみたら，Record.jsなるものがあるようだ．
https://github.com/mattdiamond/Recorderjs
Recorder.jsをダウンロードして
これをhtmlに組み込んでみる．

けど，最初やってみたら，recorder.jsでエラーでた．
CDNがあるようなので，そちらでやったらうまく行った．

In [52]:
%%writefile static/index.html

<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>
    <!-- <script src="recorder.js"></script> --> 
    
</head>
<body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording" disabled>録音停止</button>
    <!-- <audio id="audioPlayback" controls></audio> -->
    <!-- <button id="uploadAudio" disabled>アップロード</button> -->
    <p><strong>文字起こし:</strong> <span id="transcription"></span></p>
    <p><strong>AIの応答:</strong> <span id="aiResponse"></span></p>

    <script>
        let audioContext;
        let recorder;
        let audioBlob;
                

        document.getElementById("startRecording").addEventListener("click", async () => {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            audioContext = new AudioContext();
            const source = audioContext.createMediaStreamSource(stream);
            recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
            recorder.record();

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = false;
        });

        document.getElementById("stopRecording").addEventListener("click", () => {
            recorder.stop();
            recorder.exportWAV((blob) => {
                audioBlob = blob;

                if (!audioBlob) {
                    console.error("No audio to upload");    
                    return;
                }

                const formData = new FormData();
                formData.append("file", audioBlob, "recorded_audio.wav");

                fetch("/upload", {
                    method: "POST",
                    body: formData,
                })
                .then((response) => response.json())
                .then((data) => {
                    document.getElementById("transcription").textContent =
                    data.text || "認識できませんでした。";
                    document.getElementById("aiResponse").textContent =
                    data.ai_response || "AIの応答なし。";
                })
                .catch((error) => {
                    console.error("Upload failed:");
                    document.getElementById("transcription").textContent = "エラーが発生しました。";
                    document.getElementById("aiResponse").textContent = "";
                });
            });

            document.getElementById("startRecording").disabled = false;
            document.getElementById("stopRecording").disabled = true;


        });
    </script>
</body>
</html>


Overwriting static/index.html


app01.pyで実行．
CORSの問題でアップロードで弾かれているようだ．．．

flask_corsを使って，サーバの側でCORS問題を無視するように設定する．

In [ ]:
!pip install flask_cors

In [40]:
%%writefile app03.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os

app = Flask(__name__, static_folder="static")  
CORS(app)

@app.route("/")
def index():
    return send_from_directory("static", "index.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)
    
    text = "test"
    ai_response = "test"
    return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Overwriting app03.py


よし，とりあえず問題は解決した．
ハマった理由は，fetchのインタフェースをフォルダ名と勘違いしていたこと．つまり，app.routeでは/uploadとしているのに，javascriptの方で/uploadsとしていた．これにより当然ながらインタフェースがないわけで４０４エラーが返されれるということになっていた．分れば馬鹿馬鹿しい勘違いやった😂

あと，デバッグ環境ではルートディレクトリがprojectになるというところもハマった😂

## その４　Speech Recognitionにかける
よし，ここからはpythonの側の処理に集中

In [54]:
%%writefile app04.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import speech_recognition as sr

app = Flask(__name__, static_folder="static")  
CORS(app)

@app.route("/")
def index():
    return send_from_directory("static", "index.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)
            
        ai_response = "test"
        return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Overwriting app04.py


ふむ．これでとりあえず，音声認識結果を返せるようになった．

## その５　 openai の　Chat＿Compelationを使う

In [55]:
%%writefile app05.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

app = Flask(__name__, static_folder="static")  
CORS(app)

@app.route("/")
def index():
    return send_from_directory("static", "index.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)

        # AIの応答
        client = OpenAI()
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": text},
            ]
        )
        ai_response = completion.choices[0].message.content
        return jsonify({"text": text, "ai_response": ai_response})

if __name__ == "__main__":
    app.run(debug=True)


Writing app05.py


ふむ．とりあえずは単発会話はできるようになった．
現時点の違和感・修正したい点は
現状だと，入力テキストとレスポンスが同時に帰ってきてしまう．
どうにか，その部分をいじれないか．レスポンスをまたつに先に入力テキストを返して，画面に表示させておいて，レスポンスが帰ってきたら，改めてそれを返すという感じ．

## その6　入力テキストとレスポンスを分けて表記できるようにする
やるとしたら，postを2回に分ける形かな？
Copilotに聞いてみたらWeb Socketを使えばできるとな．．．
とりあえずやってみるか．

In [ ]:
!pip install flask-socketio

In [5]:
%%writefile app06.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading

load_dotenv()

app = Flask(__name__, static_folder="static")  
CORS(app)
socketio = SocketIO(app)

@app.route("/")
def index():
    return send_from_directory("static", "index06.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    threading.Thread(target=get_ai_response, args=(text,)).start()
    
    return response

def get_ai_response(text):
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": text},
        ]
    )
    ai_response = completion.choices[0].message.content
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response})

if __name__ == "__main__":
    socketio.run(app, debug=True)

Overwriting app06.py


In [7]:
%%writefile static/index06.html

<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    
</head>
<body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording" disabled>録音停止</button>
    <!-- <audio id="audioPlayback" controls></audio> -->
    <!-- <button id="uploadAudio" disabled>アップロード</button> -->
    <p><strong>文字起こし:</strong> <span id="transcription"></span></p>
    <p><strong>AIの応答:</strong> <span id="aiResponse"></span></p>

    <script>
        let audioContext;
        let recorder;
        let audioBlob;

        document.addEventListener("DOMContentLoaded", () => {
            const socket = io();

            socket.on('ai_response', (data) => {
                document.getElementById("aiResponse").textContent = data.ai_response;
            });
        });        

        document.getElementById("startRecording").addEventListener("click", async () => {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            audioContext = new AudioContext();
            const source = audioContext.createMediaStreamSource(stream);
            recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
            recorder.record();

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = false;
        });

        document.getElementById("stopRecording").addEventListener("click", () => {
            recorder.stop();
            recorder.exportWAV((blob) => {
                audioBlob = blob;

                if (!audioBlob) {
                    console.error("No audio to upload");    
                    return;
                }

                const formData = new FormData();
                formData.append("file", audioBlob, "recorded_audio.wav");

                fetch("/upload", {
                    method: "POST",
                    body: formData,
                })
                .then((response) => response.json())
                .then((data) => {
                    document.getElementById("transcription").textContent =
                    data.text || "認識できませんでした。";
                })
                .catch((error) => {
                    console.error("Upload failed:");
                    document.getElementById("transcription").textContent = "エラーが発生しました。";
                    document.getElementById("aiResponse").textContent = "";
                });
            });

            document.getElementById("startRecording").disabled = false;
            document.getElementById("stopRecording").disabled = true;


        });
    </script>
</body>
</html>


Overwriting static/index06.html


ふむ．入力とレスポンスを別々に表記できるようになった．

## その７　継続的な会話を出来るようにする．
とりあえず，フロントエンドでログを記載する部分については別に考えて，まずはパイソンで動かす

In [8]:
%%writefile app07.py
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading

load_dotenv()

app = Flask(__name__, static_folder="static")  
CORS(app)
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

@app.route("/")
def index():
    return send_from_directory("static", "index06.html")

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    threading.Thread(target=get_ai_response, args=(text,)).start()
    
    return response

def get_ai_response(text):
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response})

if __name__ == "__main__":
    socketio.run(app, debug=True)

Writing app07.py


ふむ．継続性のある会話もできるようになった．
じゃあ次は，TTSを組み込みたいね．

調べたらVoice VoxであればAPIが使えるとのこと．ただ，このAPIはあくまでローカルサーバで動くものになっている．
ローカルでどれくらい動作時間かかるんやろ？？
とりあえず試すか・・・．

## その8　音声合成機能を使ってみる
VoiceVox GUIを立ち上げておく．これによりローカルにVoiceVoxが立ち上がりAPIが使える．
そのまま関数を作ってくれている人がいたので拝借

https://zenn.dev/zenn24yykiitos/articles/fff3c954ddf42c

In [9]:
!pip install requests


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%%writefile app08.py
from flask import Flask, request, jsonify, send_from_directory, send_file
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading
import requests
import json
import time

#　環境変数の読み込み
load_dotenv()

# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#--------------------------------------------------

# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    return send_from_directory("static", "index08.html")

# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", audio_file.filename)
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            print(text)

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    threading.Thread(target=get_ai_response, args=(text,)).start()
    
    return response

# 音声ファイルを提供するエンドポイント
@app.route("/audio/<filename>")
def get_audio(filename):
    return send_file(os.path.join("uploads",filename))


#--------------------------------------------------

# AIの応答を取得する関数 
def get_ai_response(text):
    
    # 現在の時刻取得
    start = time.time()

    # OpenAIのAPIを呼び出してAIの応答を取得
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})

    # 処理時間の計算
    ai_time = time.time() - start
    print(f"処理時間: {ai_time} [sec]") 

    # 音声合成
    filename = synthesize_voice(ai_response)

    # 処理時間の計算
    voice_time = time.time() - start - ai_time
    print(f"音声合成時間: {voice_time} [sec]")
    
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response, 'audio': filename})



# 音声合成を行なう関数
def synthesize_voice(text, speaker=1, filename="uploads/output.wav"):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'http://localhost:50021/audio_query', params=query_payload)

    if query_response.status_code != 200:
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'http://localhost:50021/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        # 音声ファイルとして保存
        with open(filename, 'wb') as f:
            f.write(synthesis_response.content)
        print(f"音声が {filename} に保存されました。")
        return "output.wav"
    else:
        print(f"Error in synthesis: {synthesis_response.text}")
        return None


if __name__ == "__main__":
    socketio.run(app, debug=True)

Writing app08.py


In [1]:
%%writefile static/index08.html

<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    
</head>
<body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording" disabled>録音停止</button>
    <!-- <audio id="audioPlayback" controls></audio> -->
    <!-- <button id="uploadAudio" disabled>アップロード</button> -->
    <p><strong>文字起こし:</strong> <span id="transcription"></span></p>
    <p><strong>AIの応答:</strong> <span id="aiResponse"></span></p>

    <script>
        let audioContext;
        let recorder;
        let audioBlob;

        document.addEventListener("DOMContentLoaded", () => {
            const socket = io();

            socket.on('ai_response', (data) => {
                document.getElementById("aiResponse").textContent = data.ai_response;
                
                // 音声ファイルを自動再生する処理
                if (data.audio) {
                    const audio = new Audio(`/audio/${data.audio}`);
                    audio.play();
                }    
            });
        });        

        document.getElementById("startRecording").addEventListener("click", async () => {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            audioContext = new AudioContext();
            const source = audioContext.createMediaStreamSource(stream);
            recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
            recorder.record();

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = false;
        });

        document.getElementById("stopRecording").addEventListener("click", () => {
            recorder.stop();
            recorder.exportWAV((blob) => {
                audioBlob = blob;

                if (!audioBlob) {
                    console.error("No audio to upload");    
                    return;
                }

                const formData = new FormData();
                formData.append("file", audioBlob, "recorded_audio.wav");

                fetch("/upload", {
                    method: "POST",
                    body: formData,
                })
                .then((response) => response.json())
                .then((data) => {
                    document.getElementById("transcription").textContent =
                    data.text || "認識できませんでした。";
                })
                .catch((error) => {
                    console.error("Upload failed:");
                    document.getElementById("transcription").textContent = "エラーが発生しました。";
                    document.getElementById("aiResponse").textContent = "";
                });
            });

            document.getElementById("startRecording").disabled = false;
            document.getElementById("stopRecording").disabled = true;


        });
    </script>
</body>
</html>


Overwriting static/index08.html


とりあえず動く形にはできた！！
ただ，どうしてもレスポンスは遅い．．．．

## その９：loggingモジュールをつかってみることにする

In [8]:
%%writefile voicecahtapp09.py
from flask import Flask, request, jsonify, send_from_directory, send_file
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading
import requests
import json
import time
import logging

#　環境変数の読み込み
load_dotenv()

# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#--------------------------------------------------
#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log"
)
#--------------------------------------------------


# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index08.html")

# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    logging.info("音声ファイルをアップロードします。")
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav")
    logging.debug(f"Saving audio file to {audio_path}")
    audio_file.save(audio_path)

    # 音声認識
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
        if __debug__: # デバッグモードの場合
            logging.debug(text)
            print(text)

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    threading.Thread(target=get_ai_response, args=(text,)).start()
    
    return response

# 音声ファイルを提供するエンドポイント
@app.route("/audio/<filename>")
def get_audio(filename):
    return send_file(os.path.join("output",filename))


#--------------------------------------------------

# AIの応答を取得する関数 
def get_ai_response(text):
    
    # 現在の時刻取得
    start = time.time()

    # OpenAIのAPIを呼び出してAIの応答を取得
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})

    # 処理時間の計算
    ai_time = time.time() - start
    logging.debug(f"処理時間: {ai_time} [sec]")
    print(f"AIレスポンス時間: {ai_time} [sec]") 

    # 音声合成
    filename = synthesize_voice(ai_response)

    # 処理時間の計算
    voice_time = time.time() - start - ai_time
    logging.debug(f"音声合成時間: {voice_time} [sec]")
    print(f"音声合成時間: {voice_time} [sec]")
    
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response, 'audio': filename})



# 音声合成を行なう関数
def synthesize_voice(text, speaker=1):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'http://localhost:50021/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'http://localhost:50021/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        # 音声ファイルとして保存
        filename = f"output_{len(messages)}.wav"
        file_path = "output/" + filename
        with open(file_path, 'wb') as f:
            f.write(synthesis_response.content)
        logging.debug(f"音声が {filename} に保存されました。")
        print(f"音声が {filename} に保存されました。")
        return filename
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        print(f"Error in synthesis: {synthesis_response.text}")
        return None


if __name__ == "__main__":
    socketio.run(app, debug=True)

Overwriting voicecahtapp09.py


## その１０：htmlをチャットログが残るようにする．

In [2]:
%%writefile voicecahtapp10.py
from flask import Flask, request, jsonify, send_from_directory, send_file
from flask_cors import CORS
import os
import speech_recognition as sr
from openai import OpenAI
from dotenv import load_dotenv
from flask_socketio import SocketIO, emit
import threading
import requests
import json
import time
import logging

#　環境変数の読み込み
load_dotenv()

# Flaskアプリケーションの作成
app = Flask(__name__, static_folder="static")  

# CORSの設定
CORS(app)

# Socket.IOの設定
socketio = SocketIO(app)

# 会話ログを保持する変数
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

#--------------------------------------------------
#loggingの設定
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filename="app.log",
    encoding="utf-8"
)
#--------------------------------------------------


# ルートパスへのリクエストを処理する
@app.route("/")
def index():
    logging.info("index.html を返します。")
    return send_from_directory("static", "index10.html")

# /upload へのリクエストを処理する
@app.route("/upload", methods=["POST"])
def upload_audio():
    logging.info("音声ファイルをアップロードします。")
    if "file" not in request.files:
        logging.error("No audio file provided")
        return jsonify({"error": "No audio file provided"}), 400
    
    audio_file = request.files["file"]
    audio_path = os.path.join("uploads", f"input_{len(messages)}.wav")
    audio_file.save(audio_path)
    logging.info(f"Saved audio file to {audio_path}")

    # 音声認識
    r = sr.Recognizer()
    start_time = time.time()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        text = r.recognize_google(audio, language="ja-JP")
    logging.info(f"音声認識結果: {text}")
    logging.info(f"音声認識時間: {time.time() - start_time} [sec]")

    # 音声認識の結果を最初に返す
    response = jsonify({"text": text})
    
    # 別スレッドでAIの応答を取得
    threading.Thread(target=get_ai_response, args=(text,)).start()
    
    return response

# 音声ファイルを提供するエンドポイント
@app.route("/audio/<filename>")
def get_audio(filename):
    logging.info(f"音声ファイル {filename} を返します。")
    return send_file(os.path.join("output",filename))


#--------------------------------------------------

# AIの応答を取得する関数 
def get_ai_response(text):
    
    # 現在の時刻取得
    start = time.time()

    # OpenAIのAPIを呼び出してAIの応答を取得
    client = OpenAI()
    messages.append({"role": "user", "content": text})
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    ai_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": ai_response})
    logging.info(f"AIの応答: {ai_response}")

    # 処理時間の計算
    ai_time = time.time() - start
    logging.info(f"AIレスポンス時間: {ai_time} [sec]")

    # 音声合成
    filename = synthesize_voice(ai_response)

    # 処理時間の計算
    voice_time = time.time() - start - ai_time
    logging.info(f"音声合成時間: {voice_time} [sec]")
    
    # WebSocketを通じてクライアントに通知
    socketio.emit('ai_response', {'ai_response': ai_response, 'audio': filename})



# 音声合成を行なう関数
def synthesize_voice(text, speaker=1):
    # 1. テキストから音声合成のためのクエリを作成
    query_payload = {'text': text, 'speaker': speaker}
    query_response = requests.post(f'http://localhost:50021/audio_query', params=query_payload)

    if query_response.status_code != 200:
        logging.error(f"Error in audio_query: {query_response.text}")
        print(f"Error in audio_query: {query_response.text}")
        return

    query = query_response.json()

    # 2. クエリを元に音声データを生成
    synthesis_payload = {'speaker': speaker}
    synthesis_response = requests.post(f'http://localhost:50021/synthesis', params=synthesis_payload, json=query)

    if synthesis_response.status_code == 200:
        # 音声ファイルとして保存
        filename = f"output_{len(messages)}.wav"
        file_path = "output/" + filename
        with open(file_path, 'wb') as f:
            f.write(synthesis_response.content)
        logging.info(f"音声が {filename} に保存されました。")
        print(f"音声が {filename} に保存されました。")
        return filename
    else:
        logging.error(f"Error in synthesis: {synthesis_response.text}")
        print(f"Error in synthesis: {synthesis_response.text}")
        return None


if __name__ == "__main__":
    logging.info("#####アプリケーションを起動します。#####")
    socketio.run(app, debug=True)

Overwriting voicecahtapp10.py


In [5]:
%%writefile static/index10.html

<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>WAV録音＆アップロード</title>
    <!-- Recorder.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/gh/mattdiamond/Recorderjs@master/dist/recorder.js"></script>

    <!-- Socket.IO を読み込む -->
    <script src="https://cdn.socket.io/4.0.0/socket.io.min.js"></script>

    <!-- marked.js を読み込む -->
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>

    <!-- cssの適用-->
    <link rel="stylesheet" href="/static/voicechatapp.css">

    
</head>
<body>
    <h1>WAV録音＆アップロード</h1>
    <button id="startRecording">録音開始</button>
    <button id="stopRecording" disabled>録音停止</button>
    <div id="chatlog"></div>

    <script>
        let audioContext;
        let recorder;
        let audioBlob;

        document.addEventListener("DOMContentLoaded", () => {
            const socket = io();

            socket.on('ai_response', (data) => {
                const markdownText = data.ai_response;
                const htmlContent = marked.parse(markdownText);
                document.getElementById("chatlog").innerHTML += `<div class="assistant">${htmlContent}</div>`;
                
                // 音声ファイルを自動再生する処理
                if (data.audio) {
                    const audio = new Audio(`/audio/${data.audio}`);
                    audio.play();
                }
                document.getElementById("startRecording").disabled = false;
                document.getElementById("stopRecording").disabled = true;                    
            });

            // Spaceキーが押されたときにstartRecordingボタンをクリック
            document.addEventListener("keydown", (event) => {
                if(document.getElementById("startRecording").disabled){ 
                    console.log("処理中のため入力はできません");
                    return;
                }
                if (event.code === "Space" && !event.repeat) {
                    document.getElementById("startRecording").click();
                }
            });

            // Spaceキーから指が離されたときにstopRecordingボタンをクリック
            document.addEventListener("keyup", (event) => {
                if(document.getElementById("stopRecording").disabled){
                    console.log("不正な録音停止操作です");
                    return;
                }
                if (event.code === "Space" && !event.repeat) {
                    document.getElementById("stopRecording").click();
                }
            });

        });        

        document.getElementById("startRecording").addEventListener("click", async () => {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            audioContext = new AudioContext();
            const source = audioContext.createMediaStreamSource(stream);
            recorder = new Recorder(source, { numChannels: 1 }); // モノラル録音
            recorder.record();

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = false;
        });

        document.getElementById("stopRecording").addEventListener("click", () => {
            recorder.stop();
            recorder.exportWAV((blob) => {
                audioBlob = blob;

                if (!audioBlob) {
                    console.error("No audio to upload");    
                    return;
                }

                const formData = new FormData();
                formData.append("file", audioBlob, "recorded_audio.wav");

                fetch("/upload", {
                    method: "POST",
                    body: formData,
                })
                .then((response) => response.json())
                .then((data) => {
                    if(data.text){
                         document.getElementById("chatlog").innerHTML += `<div class="user">${marked.parse(data.text)}</div>`;
                    }
                    else console.log("Error: 音声を認識できませんでした。");
                })
                .catch((error) => {
                    console.error("Upload failed:");
                });
            });

            document.getElementById("startRecording").disabled = true;
            document.getElementById("stopRecording").disabled = true;
        });


    </script>
</body>
</html>


Overwriting static/index10.html


概ね上手く行くようにできた．
後の問題は，AIからのレスポンスがマークダウン形式になっているのをうまく表記することかな．

-> 対応した　marked.jsなんてのがあるんや．これ使えば簡単！

https://zenn.dev/zenn24yykiitos/articles/f3e983fe650e08

これを参考に，フロントエンドからキャラクターを選べるようにする．